In [9]:
from tensorflow.keras.applications import ResNet101, VGG19, ResNet101V2, ResNet50, ResNet50V2, VGG16, InceptionResNetV2, InceptionV3, MobileNetV3Small
import tensorflow.keras,os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D , Flatten, Dropout, ZeroPadding2D, GlobalAveragePooling2D, DepthwiseConv2D, Concatenate, add
from tensorflow.keras import optimizers
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

In [2]:
img_shape = (224, 224, 3)
input_img = Input(shape=img_shape, name='main_input')

In [3]:
model = InceptionV3(include_top=False, weights=None, input_shape=img_shape)(input_img)

In [4]:
model = Flatten()(model)
out = Dense(units=1, activation="sigmoid")(model)
model0 = Model(inputs=input_img, outputs=out)
model0.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
model0.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 224, 224, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51201     
Total params: 21,853,985
Trainable params: 21,819,553
Non-trainable params: 34,432
_________________________________________________________________


In [8]:
model0.save("./tn_model/inceptionv3.h5")

In [11]:
mobile = MobileNetV3Small(include_top=True, weights=None, input_shape=img_shape)
mobile.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, 112, 112, 16) 432         rescaling_1[0][0]                
__________________________________________________________________________________________________
Conv/BatchNorm (BatchNormalizat (None, 112, 112, 16) 64          Conv[0][0]                       
___________________________________________________________________________________

In [12]:
mobile.save("./tn_model/mobilev3small.h5")

D:\Program Files\python38\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [6]:
model1 = Sequential([model])
model1.add(Flatten())
model1.add(Dense(units=1, activation="sigmoid"))
model1.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
model1.trainable = False
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51201     
Total params: 21,853,985
Trainable params: 0
Non-trainable params: 21,853,985
_________________________________________________________________


In [6]:
###########################################################
# 데이터 전처리
###########################################################


train_datagen = ImageDataGenerator(rescale=1./255)  # 모든 이미지를 1/255로 스케일을 조정합니다
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = "./dataset/train2"
validation_dir = './dataset/vali2'

batch_size = 8
# batch_size = 64


train_generator = train_datagen.flow_from_directory( \
        train_dir,                  # 타깃 디렉터리
        target_size=(224, 224),     # 모든 이미지를 224 × 224 크기
        batch_size=batch_size,
        class_mode='binary')        #

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)  #(20, 150, 150, 3)
    print('배치 레이블 크기:', labels_batch.shape) #(20,)
    break

Found 802 images belonging to 2 classes.
Found 286 images belonging to 2 classes.
배치 데이터 크기: (8, 224, 224, 3)
배치 레이블 크기: (8,)


In [7]:
steps_per_epochs = 50 # 학습데이터의 총 개수 / 배치 사이즈 (소수점 올림)
epochs = 10

history = model0.fit(
      train_generator ,
      steps_per_epoch=steps_per_epochs  ,
      epochs=epochs ,
      validation_data=validation_generator,
      validation_steps=10)

Epoch 1/10
17/50 [=========>....................] - ETA: 7s - loss: 0.8738 - accuracy: 0.5956

KeyboardInterrupt: 

In [ ]:
###########################################################
# 학습 결과 정확도 및 비용 그래프
###########################################################
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')

plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
model.save("./tn_model/inception_v3.h5")